In [230]:
import numpy as np
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch

In [231]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )

    def forward(self, x):
        return x + self.block(x)

In [232]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        # (int_channel, out_channel, kernel_size, stride, padding, output_padding)
        self.conv1 = nn.Conv2d(3,64,3,2,1)
        self.norm1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
            
        self.conv2 = nn.Conv2d(64,128,3,2,1)
        self.norm2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
            
        self.conv3 = nn.Conv2d(128,256,3,2,1)
        self.norm3 = nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU()
            
        self.conv4 = nn.Conv2d(256,512,3,2,1)
        self.norm4 = nn.BatchNorm2d(512)
        self.relu4 = nn.ReLU()
            
        self.conv5 = nn.Conv2d(512,1024,3,2,1)
        self.norm5 = nn.BatchNorm2d(1024)
        self.relu5 = nn.ReLU()
        
        self.res1 = ResidualBlock(1024)

        self.convtrans1 = nn.ConvTranspose2d(1024,512,3,2,padding = 1,output_padding=1)
            
        self.convtrans2 = nn.ConvTranspose2d(512,256,3,2,padding = 1,output_padding=1)
            
        self.convtrans3 = nn.ConvTranspose2d(256,128,3,2,padding = 1,output_padding=1)
          
        self.convtrans4 = nn.ConvTranspose2d(128,64,3,2,padding = 1,output_padding=1)
            
        self.convtrans5 = nn.ConvTranspose2d(64,32,3,2,padding = 1,output_padding=1)
        
        self.out1 = nn.Conv2d(32,32,3,1,1)
        
        self.out2 = nn.Conv2d(32,3,3,1,1)
        
    def forward(self, x):
        y = self.conv1(x)
        y = self.norm1(y)
        y_conv1 = self.relu1(y)
            
        y = self.conv2(y_conv1)
        y = self.norm2(y)
        y_conv2 = self.relu2(y)
            
        y = self.conv3(y_conv2)
        y = self.norm3(y)
        y_conv3 = self.relu3(y)
            
        y = self.conv4(y_conv3)
        y = self.norm4(y)
        y_conv4 = self.relu4(y)
            
        y = self.conv5(y_conv4)
        y = self.norm5(y)
        y = self.relu5(y)
        
        y = self.res1(y)

        y1 = self.convtrans1(y)
        z1 = y_conv4
        y = torch.cat((z1,y1),dim=0)
        
            
        y2 = self.convtrans2(y)
        z2 = y_conv3
        y = torch.cat((z2,y2),dim=0)
            
        y3 = self.convtrans3(y)
        z3 = y_conv2
        y = torch.cat((z3,y3),dim=0)
            
        y4 = self.convtrans4(y)
        z4 = y_conv1
        y = torch.cat((z4,y4),dim=0)
            
        y = self.convtrans5(y)
        
        y = self.out1(y)
        y = self.out2(y)

        return y

In [234]:
from torchsummary import summary
reg = Regressor()
print(reg.forward)
#print(reg)
summary(reg.cuda(), (3, 128, 128))

<bound method Regressor.forward of Regressor(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (conv5): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm5): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu5): ReLU()
  (res1): Re